## Pool Simulator!

We're making a retro pool simulator in which a user can input an angle and force to the cue ball and play a turn based game of pool.

In [4]:
# imports

import numpy as np
import matplotlib.pyplot as plt
#from future import annotations

In [5]:
# Ball class

def Ball():
    position: tuple[float, float]
    velocity: float
    team: str

    
    def __init__(x_0,y_0, team):
        position = (x_0,y_0)
        velocity = 0.0
        team = team
    
    def ballCollision():
        ...
    
    def wallCollision():
        ...
        

In [6]:
#physics simulator

def State():
    allBalls: list | dict
    tableParams = list | dict
    score: int
    turn: int

    def __init__(): 
        ...

def Simulator():
    state: State
    inputs: list | dict
        
    def __init__():
        ...

In [ ]:
#new game

In [4]:
#plotting trajectories

In [ ]:
'''stretch goals: 

 - non-regulation non-euclidean pool tables, using a table class that takes in the number of sides and their lengths'''